# Code to analyze and visualize scRNAseq data - R

## A. Cluster cells and find defining genes per cluster
Packages required: cellrangerRkit, NBClust, edgeR, GOplot, devtools

References: 

http://cf.10xgenomics.com/supp/cell-exp/cellrangerrkit-PBMC-vignette-knitr-2.0.0.pdf

https://www.bioconductor.org/packages/release/bioc/vignettes/edgeR/inst/doc/edgeRUsersGuide.pdf

Locate file path that contains files produced from 10x sequencing (including gene BC matrix and mouse genome info), set as working directory

In [ ]:
setwd("~your/file/path")

Load gene BC matrix (contains info about sample barcodes, expression counts per cell) and analysis results (10X performs some preliminary analysis)

In [ ]:
gbm<-load_cellranger_matrix("./")
analysis_results<-load_cellranger_analysis_results

### 1. Find optimal number of clusters for the dataset
Use coordinates for all cells in tSNE plot: 'tsne_kmeans.csv' (in Github repository)

In [ ]:
tsne_kmeans<-read.csv('tsne_kmeans.csv')
fviz_nbclust(tsne_kmeans,kmeans,method="silhouette",k.max=15)

### 2. Find top defining genes per cluster
Use 9 K-means clusters based on silhouette analysis. Can change 'n' to the number of defining genes required. Output file will be in the working directory

In [ ]:
cells_to_plot<-order_cell_by_clusters(gbm, analysis_results$clustering$kmeans_9_clusters$Cluster)
prioritized_genes<-prioritize_top_genes(gbm, analysis_results$clustering$kmeans_9_clusters$Cluster, "sseq", min_mean=0.5)
write_cluster_specific_genes(prioritized_genes, output_folder, n_genes=50)

### 3. Find genes differentially expressed between conditions
Use new .csv file that only contains experimental condition and gene expression data from a single cluster. See 'Cluster1_edgeR.txt' (in Github repository). The stringency set in rowSums is based on trial and error for similar experiments in our lab. Can change 'n' to the number of DEGs required.

In [ ]:
x<-read.delim("Cluster1_edgeR.txt",row.names="symbol")
group<-factor(c(1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
                2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2))
y<-DGEList(counts=x,group=group)
keep<-rowSums(cpm(y)>5)>=8
y<-y[keep, , keep.lib.sizes=FALSE]
y<-calcNormFactors(y)
design<-model.matrix(~group)
y<-estimateDisp(y,design)
fit<-glmQLFit(y,design)
qlf1vs2<-glmQLFTest(fit,coef=2)
topgenesQLFtest<-topTags(qlf1vs2, n=2000)
write.csv(topgenesQLFtest,"topgenesQLFtest.csv")